# Working with  PostgreSQL in Jupyter Notebooks

## Installing Libaries 
You can connect to PostgreSQL from Jupyter/Python by running following commands. We will use psycopg2 for working with 
PostgreSQL. We will use Pandas and Pprint for showing our output in a better way. 

In [1219]:
#You don't need to run this on Tux, on your local machine uncomment these lines and run it once.
#!pip install psycopg2
#!pip install pprint
#!pip install pandas



## Importing Libaries
After the installation we need to import all the necessary libraries into the project.

In [1220]:
import psycopg2  
import pprint
import pandas as pd #show data as tables

#For Interactive design. 
from ipywidgets import *
from IPython.display import display
from IPython.display import clear_output



## DBUtils Class
This class will handle all the database functions. You don't need to change this class. 

In [1221]:


class DBUtils:


    
   # Open a database connection.
   # 
   # @param user
   # @param pass
   # @param dbSID
   # @param host
   # @return connection
    @staticmethod  
    def  openDBConnection(dbUser,dbPass,dbSID,dbHost,dbPort):
        conn = psycopg2.connect("dbname={} user={} password={} host={} port={}".format(dbSID,dbUser,dbPass,dbHost,dbPort))  
        return conn;

        
   # Test a database connection.
   # 
   # @param conn
   # @return current date and time if the connection is open.  Otherwise an exception will be thrown.
    @staticmethod
    def testConnection(conn):
        cur = conn.cursor();
        cur.execute("select now() as res")
        res = ""
        row = cur.fetchone()
        while row is not None:
            res = "Servus: " + str(row[0])
            row= cur.fetchone()
        cur.close()
        return res

    
   # Close the database connection.
   # 
   # @param conn   
    @staticmethod
    def closeConnection(conn):
        conn.close()
        
    
   # Execute an update or a delete query.
   # @param conn
   # @param query
    @staticmethod
    def executeUpdate(conn,query,parameters=None):
        cur = conn.cursor()
        cur.execute(query,parameters)
        conn.commit()
        cur.close()


   # Get a variable that is returned as a result of a query.
   # @param conn
   # @param query
   # @return result
    @staticmethod
    def getVar(conn,query,parameters=None):
        cur = conn.cursor()
        cur.execute(query,parameters)
        result = cur.fetchone()[0]
        cur.close()
        return result
        
   # Get a row that is returned as a result of a query.
   # @param conn
   # @param query
   # @return result
    @staticmethod
    def getRow(conn,query,parameters=None):
        cur = conn.cursor()
        cur.execute(query,parameters)
        result = cur.fetchone()
        cur.close()
        return result
        
    
   # Get  all rows that is returned as a result of a query.
   # @param conn
   # @param query
   # @return result
    @staticmethod
    def getAllRows(conn,query, parameters=None):
        cur = conn.cursor()
        cur.execute(query, parameters)
        result = cur.fetchall()
        cur.close()
        return result
        
    

## Registrar Class
An implementation of the Registrar

In [1258]:
# Read the java properties file.
# 
# @param boundle address
# @return a dictionary containing the connection information 
def getBundle(filepath, sep='=', comment_char='#'):
    props = {}
    with open(filepath, "rt") as f:
        for line in f:
            l = line.strip()
            if l and not l.startswith(comment_char):
                key_value = l.split(sep)
                key = key_value[0].strip()
                value = sep.join(key_value[1:]).strip().strip('"') 
                props[key] = value 
    return props


class Registrar:
    
    def __init__(self):
        self._conn = None
        self._bundle = None
    
    
   # Open a database connection.
   # 
   # @param boundle address
   # @return connection  
    def  openDBConnectionWithBundle(self, bundle):
        prop =getBundle(bundle)
        return self.openDBConnection(prop['dbUser'],prop['dbPass'],prop['dbSID'],prop['dbHost'],prop['dbPort']) 
    
   # Open the database connection.
   # @param dbUser
   # @param dbPass
   # @param dbSID
   # @param dbHost
   # @return
    def openDBConnection(self, dbUser,dbPass,dbSID,dbHost,port):
        if (self._conn != None):
            self.closeDBConnection()
        try:
            self._conn = DBUtils.openDBConnection(dbUser, dbPass, dbSID, dbHost, port)
            res = DBUtils.testConnection(self._conn)
        except psycopg2.Error as e: 
            print (e)
        return res


   # Close the database connection.
    def closeConnection(self):
        try:
            DBUtils.closeConnection(self._conn)
        except psycopg2.Error as e: 
            print (e)
            

   # Register a new student in the database.
   # @param newStudent
   # @return
    def registerUser(self, newUser):
        try :
            query = """
                insert into users (first_name, last_name, password, user_type, username) values (%s,%s,%s,%s,%s) 
            """
            print ("User has been Registered")
            DBUtils.executeUpdate(self._conn, query,(newUser._first_name, newUser._last_name, newUser._password, newUser._user_type, newUser._username))
        except psycopg2.Error as e: 
                print (e)
    
        return newUser
    
    
    def studentProfile(self, username):
        try :
            row = []
            query = "select concat(u.first_name, ' ', u.last_name) as name, u.username, u.user_type, title, string_agg(category_name, ', ') as categories " \
            "from users u " \
            "left outer join course_users cu on u.username = cu.username " \
            "left outer join course_categories cc on cu.course_id = cc.course_id " \
            "left outer join courses c on c.course_id = cu.course_id " \
            "where u.username ='" + str(username) + "' " \
            "group by c.title,c.course_id,u.first_name,u.last_name,u.username,u.user_type"
            row = DBUtils.getAllRows(self._conn, query)
        except psycopg2.Error as e: 
                print (e)
    
        return row 
    
    def createCourse(self, course, video, document, courseCategoryList):
        try :
            course_id = 1 + int((DBUtils.getVar(self._conn, "select max(course_id) from Courses")) or 0)
            course.setCourseId(course_id)
            query = """
                insert into courses (course_id, title, ratings, tutor) values (%s,%s,%s,%s) 
            """
            DBUtils.executeUpdate(self._conn, query,(course.getCourseId(),course.getTitle(),course.getRating(), course.getTutor()))
        
            video_id = 1 + int((DBUtils.getVar(self._conn, "select max(video_id) from Videos")) or 0)
            video.setVideoId(video_id)
            video.setCourseId(course_id)
            query = """
                insert into videos (video_id, title, course_id) values (%s,%s,%s) 
            """
            DBUtils.executeUpdate(self._conn, query,(video.getVideoId(),video.getTitle(),video.getCourseId()))
        
        
        
            doc_id = 1 + int((DBUtils.getVar(self._conn, "select max(doc_id) from Documents")) or 0)
            document.setDocId(doc_id)
            document.setCourseId(course_id)
            query = """
                insert into documents (doc_id, title, course_id) values (%s,%s,%s) 
            """
            DBUtils.executeUpdate(self._conn, query,(document.getDocId(),document.getTitle(),document.getCourseId()))
        
               
            
            for courseCategory in courseCategoryList:
                courseCategory.setCourseId(course_id)
                query = """
                    insert into course_categories (course_id, category_name) values (%s,%s) 
                """
                DBUtils.executeUpdate(self._conn, query,(courseCategory.getCourseId(),courseCategory.getCatgoryName()))
        
        except psycopg2.Error as e: 
                print (e)
        return 2    

    def courseListing(self, username):
        try :
            unenrolled = []
            enrolled = []
            query_unenrolled = "select c.course_id,c.title, c.ratings, concat(t.first_name, ' ', t.last_name) as tutor, string_agg(cc.category_name, ', ') as categories" \
                " from courses c" \
                " left outer join course_users cu on (cu.course_id = c.course_id and cu.username = '" + username + "')" \
                " left outer join course_categories cc on cc.course_id = c.course_id" \
                " join users t on t.username = c.tutor" \
                " where cu.course_id is null" \
                " group by c.course_id,c.title, c.ratings, t.first_name, t.last_name"
            unenrolled = DBUtils.getAllRows(self._conn, query_unenrolled)
            c_u = pd.DataFrame(columns=["course_id", "course_title", "course_rating", "Tutor", "Category"])
            
            joinButton = widgets.Button(
                description='Join',
                disabled=False,
            )
            
            courses_join = {}
            for i in unenrolled:
                c_u.loc[c_u.shape[0]] = i
                courses_join[i[1]] = i[0]

            
            joined_courses = {}
            query_enrolled = "select c.course_id,c.title, c.ratings, concat(t.first_name, ' ', t.last_name) as tutor, string_agg(cc.category_name, ', ') as categories" \
            " from users u" \
            " join course_users cu on cu.username = u.username" \
            " join courses c on c.course_id = cu.course_id" \
            " left outer join course_categories cc on cc.course_id = c.course_id" \
            " join users t on t.username = c.tutor" \
            " where u.username ='" + str(username) + "' " \
            " group by c.course_id,c.title, c.ratings, t.first_name, t.last_name"
            enrolled = DBUtils.getAllRows(self._conn, query_enrolled)
            c_e = pd.DataFrame(columns=["course_id", "course_title", "course_rating", "Tutor", "Category"])
            for i in enrolled:
                c_e.loc[c_e.shape[0]] = i
                joined_courses[i[1]] = i[0]
        except psycopg2.Error as e: 
                print (e)
    
        return c_e, c_u, courses_join, joined_courses 
    
    
    def enrollCourse(self, user_enroll):
        try:            
            query = "insert into course_users (course_id, username) values (%s, %s)"
            DBUtils.executeUpdate(self._conn, query,(user_enroll.getCourseId(),user_enroll.getUsername()))
        except psycopg2.Error as e: 
            print (e)
        
        return 1
        
    def showCourse(self, course_id, username):
        test_result = None
        review_list = []
        student_list = []
        try:
            course_query = "select c.title, c.ratings, concat(t.first_name, ' ', t.last_name) as tutor, avg(tr.result_score) as average_score" \
            " from courses c" \
            " left outer join test_results tr on (tr.username = '" + username +"' and tr.course_id='" + str(course_id) + "')" \
            " join users t on c.tutor = t.username" \
            " where c.course_id = '" + str(course_id) + "'" \
            " group by c.title, c.ratings, t.first_name, t.last_name, c.course_id"
            course = DBUtils.getRow(self._conn, course_query)

            document_query = "select title" \
            " from documents d" \
            " where d.course_id = '" + str(course_id) + "'"
            document = DBUtils.getAllRows(self._conn, document_query)

            video_query = "select title" \
            " from videos v" \
            " where v.course_id = '" + str(course_id) + "'"
            video = DBUtils.getAllRows(self._conn, video_query)
            
            test_query = "select t.title, t.description, t.pass_percentage, tr.result_score," \
            " case" \
            " when t.pass_percentage <= tr.result_score then 'pass'" \
            " else 'fail'" \
            " end" \
            " from test t " \
            " join test_results tr on (tr.test_id = t.test_id and tr.username = '" + username + "')" \
            " where tr.course_id = '" + str(course_id) + "'"
            test_result = DBUtils.getAllRows(self._conn, test_query)
            
            review_query = "select concat(u.first_name, ' ', u.last_name) as student, cu.review_rating, cu.review_comment" \
            " from course_users cu" \
            " join users u on u.username = cu.username" \
            " where cu.course_id = '" + str(course_id) + "'"
            review = DBUtils.getAllRows(self._conn, review_query)

            for i in review:
                if i[1] is not None:
                    review_list.append(i)
                student_list.append(i[0])
            
        except psycopg2.Error as e: 
            print (e)
        return course, document, video, test_result, review_list, student_list    
            
    
    def addTermsDynamicSQL(self, terms):
        for i in range(len(terms)):
            term = terms[i]
        try:
            query = "insert into Terms values ('" + term + "')";
            DBUtils.executeUpdate(_conn, query);
        except psycopg2.Error as e: 
            print (e)
            
    def addTermsPreparedStatement(self, terms):
        raise Exception("Not Supported in psycopg2")
        
    def showAsTable(self, rows, columnList):
        df = pd.DataFrame(columns=columnList)
        for i in rows:
            df.loc[df.shape[0]] = i
        return df
    


## User Class
 An implementation of the User class.


In [1259]:
# An implementation of the User class.
class User:
    # Constructor
    # @param id
    # @param name
    # @param gpa
    def __init__(self, first_name, last_name, password, user_type, username=None):
        self._first_name = first_name
        self._last_name = last_name
        self._username = username
        self._password = password
        self._user_type = user_type


    def getUsername(self):
        return self._username
        
    def getFirstName(self):
        return self._first_name
        
    def getLastName(self):
        return self._last_name
    
    def getUserType(self):
        return self._user_type;
    
    def getPassword(self):
        return self._password
    
    def setUserName(self, username):
        self._username = username
        
    def setFirstName(self, first_name):
        self._first_name = first_name
        
    def setLastName(self, last_name):
        self._last_name = last_name
    
    def setPassword(self, password):
        self._password = password
        
    def setUserType(self, user_type):
        self._user_type = user_type
        
    
    # Generate a string representation of the student.
    # @return string representation
    def __str__(self):
        return str(self._username) + " " + str(self._user_type) + " " + str(self._password) + " " + str(self._first_name) + " " + str(self._last_name) 

    
    @staticmethod
    def showAsTable(rows):
        df = pd.DataFrame(columns=["username","first_name","last_name","password","user_type"])
        for i in rows:
            df.loc[df.shape[0]] = i
        return df


## Course Class

In [1260]:
# An implementation of the Student class.
class Course:
    # Constructor
    # @param id
    # @param name
    # @param gpa
    def __init__(self, title, tutor, ratings=None, course_id=None):
        self._course_id = course_id
        self._title = title
        self._ratings = ratings
        self._tutor = tutor

    # Set the student's gpa.
    # @param gpa

    def getCourseId(self):
        return self._course_id

    def getTitle(self):
        return self._title

    def getRating(self):
        return self._ratings

    def getTutor(self):
        return self._tutor

    def setCourseId(self, course_id):
        self._course_id = course_id

    def setTitle(self, title):
        self._title = title

    def setRating(self, ratings):
        self._ratings = ratings

    def setTutor(self, tutor):
        self._tutor = tutor

    # Generate a string representation of the student.
    # @return string representation
    def __str__(self):
        return str(self._course_id + " " + self._title + " " + self._ratings + " " + self._tutor)


    # Show a list of students as panda table.
    # @return panda dataframe
    @staticmethod
    def showAsTable(rows):
        df = pd.DataFrame(columns=["course_id", "title", "ratings", "tutor"])
        for i in rows:
            df.loc[df.shape[0]] = i
        return df



## Course Category Class

In [1261]:
# An implementation of the Student class.
class Course_Category:
    # Constructor
    # @param id
    # @param name
    # @param gpa
    def __init__(self, category_name, course_id=None):
        self._category_name = category_name
        self._course_id = course_id


    def getCourseId(self):
        return self._course_id

    def getCatgoryName(self):
        return self._category_name

    def setCourseId(self, course_id):
        self._course_id = course_id

    def setCategoryName(self, category_name):
        self._category_name = category_name

    # Generate a string representation of the student.
    # @return string representation
    def __str__(self):
        return str(self._course_id + " " + self._category_name)


    # Show a list of students as panda table.
    # @return panda dataframe
    @staticmethod
    def showAsTable(rows):
        df = pd.DataFrame(columns=["course_id", "category_name"])
        for i in rows:
            df.loc[df.shape[0]] = i
        return df
    


## Course User Class

In [1262]:
# An implementation of the Student class.
class Course_User:
    # Constructor
    # @param id
    # @param name
    # @param gpa
    def __init__(self, course_id, username, review_rating = None, review_comment = None):
        self._username = username
        self._review_rating = review_rating
        self._review_comment = review_comment
        self._course_id = course_id


    def getCourseId(self):
        return self._course_id

    def getUsername(self):
        return self._username

    def getReviewRating(self):
        return self._review_rating

    def getReviewComment(self):
        return self._review_comment

    def setCourseId(self, course_id):
        self._course_id = course_id

    def setUsername(self, username):
        self._username = username

    def setReviewRating(self, review_rating):
        self._review_rating = review_rating

    def setReviewComment(self, review_comment):
        self._review_comment = review_comment

    # Generate a string representation of the student.
    # @return string representation
    def __str__(self):
        return str(self._course_id + " " + self._username + " " + self._review_rating + " " + self._review_comment)


    # Show a list of students as panda table.
    # @return panda dataframe
    @staticmethod
    def showAsTable(rows):
        df = pd.DataFrame(columns=["course_id", "username", "review_rating", "review_comment"])
        for i in rows:
            df.loc[df.shape[0]] = i
        return df


## Video Class

In [1263]:
# An implementation of the Student class.
class Video:
    # Constructor
    # @param id
    # @param name
    # @param gpa
    def __init__(self, title, course_id=None, video_id=None):
        self._video_id = video_id
        self._title = title
        self._course_id = course_id

    def getTitle(self):
        return self._title
    
    def getVideoId(self):
        return self._video_id
    
    def getCourseId(self):
        return self._course_id
    
    def setCourseId(self, course_id):
        self._course_id = course_id
        
    def setVideoId(self, video_id):
        self._video_id = video_id
        
    def setTitle(self, title):
        self._title = title
        
        
    # Generate a string representation of the student.
    # @return string representation
    def __str__(self):
        return str(self._video_id + " " + self._title + " " + self._course_id)

    # Show a list of students as panda table.
    # @return panda dataframe
    @staticmethod
    def showAsTable(rows):
        df = pd.DataFrame(columns=["course_id", "title", "video_id"])
        for i in rows:
            df.loc[df.shape[0]] = i
        return df

## Document Table

In [1264]:
# An implementation of the Student class.
class Document:
    # Constructor
    # @param id
    # @param name
    # @param gpa
    def __init__(self, title=None, course_id=None, doc_id=None):
        self._docid = doc_id
        self._title = title
        self._courseid = course_id

    # Get the name of the student.
    #  @return name
    def getDocId(self):
        return self._docid

    # Get the id of the student.
    #  @return id
    def getTitle(self):
        return self._title

    def getCourseId(self):
        return self._courseid

    # Set the student's name.
    # @param name
    def setDocId(self, docid):
        self._docid = docid

    # Set the student's id.
    # @param id
    def setTitle(self, title):
        self._title = title

    def setCourseId(self, courseid):
        self._courseid = courseid

    # Generate a string representation of the student.
    # @return string representation
    def __str__(self):
        return str(self._docid) + " " + str(self._title) +  " " + str(self._courseid)

    # Show a list of students as panda table.
    # @return panda dataframe
    @staticmethod
    def showAsTable(rows):
        df = pd.DataFrame(columns=["doc_id", "title", "course_id"])
        for i in rows:
            df.loc[df.shape[0]] = i
        return df



## User Registration Form


In [1265]:

firstName = widgets.Text(
    value='',
    placeholder='First Name',
    disabled=False
)

lastName = widgets.Text(
    value='',
    placeholder='Last Name',
    disabled=False
)

username = widgets.Text(
    value='',
    placeholder='username',
    disabled=False
)

password = widgets.Text(
    value='',
    placeholder='password',
    disabled=False
)


user_type = widgets.RadioButtons(
    options=['student', 'tutor'],
    disabled=False
)


button = widgets.Button(description="Register")


#Display Widgets
display(lastName)
display(firstName)
display(password)
display(username)
display(user_type)
display(button)



#Click Function
def run_queries(sender):
    reg = Registrar()
    responce = reg.openDBConnectionWithBundle("PgBundle.properties")
    print (responce)
    newUser = User(firstName.value, lastName.value, password.value, user_type.value, username.value)

    newUser = reg.registerUser(newUser)
    #print("\nRegistered a new user: " + str(newUser));

    #roster = reg.getRoster();
    reg.closeConnection()
    #table=User.showAsTable(roster)
    #display(table)


#Click Handlers
button.on_click(run_queries)




Text(value='', placeholder='Last Name')

Text(value='', placeholder='First Name')

Text(value='', placeholder='password')

Text(value='', placeholder='username')

RadioButtons(options=('student', 'tutor'), value='student')

Button(description='Register', style=ButtonStyle())

## Student Profile page

In [1266]:

username = widgets.Text(
    value='',
    placeholder='Username for eg. hp8, hg, rw32',
    disabled=False
)

button = widgets.Button(description="Profile")

display(username)
display(button)



def get_profile(sender):
    clear_output()
    display(username)
    display(button)
    reg = Registrar()
    responce = reg.openDBConnectionWithBundle("PgBundle.properties")
    print (responce)
    result = reg.studentProfile(username.value)
    if not result:
        print ("Hello Muggle, the username doesn't exist.")
        return
    
    display(widgets.HTML(
    value="<H1>" + result[0][0] + "</H1><br>" \
        "Username: " + str(result[0][1]) + "<br>" \
        "User Type: " + str(result[0][2]) + "<br>" 
    ))

    new_list = []
    for x in result:
        new_list.append(x[3:])       

    display(reg.showAsTable(new_list,['Course','Categories']))
    reg.closeConnection()

button.on_click(get_profile)



Text(value='', placeholder='Username for eg. hp8, hg, rw32')

Button(description='Profile', style=ButtonStyle())

## Course Creation Page

In [1267]:
tutor= widgets.Text(
    value='',
    placeholder='Tutor username',
    disabled=False
)

title = widgets.Text(
    value='',
    placeholder='course name',
    disabled=False
)

categories = widgets.SelectMultiple(
    options=['magic', 'muggles', 'spells'],
    value=['magic'],
    #rows=10,
    description='Categories',
    disabled=False
)

video = widgets.Text(
    value='',
    placeholder='video',
    disabled=False
)

document= widgets.Text(
    value='',
    placeholder='document',
    disabled=False
)
   
button = widgets.Button(description="Create Course")

display(tutor)
display(title)
display(video)
display(categories)
display(document)
display(button)


def create_course(sender):
    reg = Registrar()
    responce = reg.openDBConnectionWithBundle("PgBundle.properties")
    
    categoryList = []
    # Creating list of course category classes
    for category in categories.value:
        categoryList.append(Course_Category(category))
    
    newCourse = Course(title.value, tutor.value, None, None)
    result = reg.createCourse(newCourse, Video(video.value), Document(document.value), categoryList)
    print("Course Created")
    reg.closeConnection()

button.on_click(create_course)

Text(value='', placeholder='Tutor username')

Text(value='', placeholder='course name')

Text(value='', placeholder='video')

SelectMultiple(description='Categories', index=(0,), options=('magic', 'muggles', 'spells'), value=('magic',))

Text(value='', placeholder='document')

Button(description='Create Course', style=ButtonStyle())

## Course Listing

In [1272]:

username = widgets.Text(
    value='',
    placeholder='Username eg: hp8, hg, rw32',
    disabled=False
)

button = widgets.Button(description="Login")

display(username)
display(button)

enroll = None
enrolled = None
unenrolled = None
show_course_dropdown = None

def show_course(sender):
    global show_course_dropdown
    clear_output()
    reg = Registrar()
    responce = reg.openDBConnectionWithBundle("PgBundle.properties")
    course_id = show_course_dropdown.value
    course, document, video, test_result, review_list, student_list = reg.showCourse(course_id, username.value)
       
    average = ""
    if course[3] is not None:
        average = str(round(course[3],2))
    
    display(widgets.HTML(
    value="<H1>" + course[0] + "</H1><br>" \
        "Rating: " + str(course[1]) + "<br>" \
        "Tutor: " + str(course[2]) + "<br>" \
        "Average Score: " + average + "<br>" \
    ))
    
    display(widgets.HTML(
    value="<b>Documents</b>",
    ))
    col_list = ['Title']
    display(reg.showAsTable(document,col_list))
    
    display(widgets.HTML(
    value="<b>Videos</b>",
    ))
    col_list = ['Title']
    display(reg.showAsTable(video,col_list))
    
    display(widgets.HTML(
    value="<b>Tests</b>",
    ))
    col_list = ['Title', 'Description', 'Pass Mark', 'Scored', 'Result']
    display(reg.showAsTable(test_result,col_list))
    
        
    display(widgets.HTML(
    value="<b>Reivews</b>",
    ))
    col_list = ['Student','Rating', 'Comment']
    display(reg.showAsTable(review_list,col_list))
    
    display(widgets.HTML(
    value="<b>Classmates</b>",
    ))
    col_list = ['Student']
    display(reg.showAsTable(student_list,col_list))
    
    reg.closeConnection()
    return 

def join_course(sender):
    global enrolled
    global unenrolled
    global enroll
    clear_output()
    print ("Successfully enrolled")
    reg = Registrar()
    responce = reg.openDBConnectionWithBundle("PgBundle.properties")
    user_enroll = Course_User(enroll.value, username.value)
    reg.enrollCourse(user_enroll)
    reg.closeConnection()
    course_listing(None)


def course_listing(sender): 
    clear_output()
    global enroll
    global enrolled
    global unenrolled
    global show_course_dropdown
    reg = Registrar()
    responce = reg.openDBConnectionWithBundle("PgBundle.properties")
    print (responce)
    enrolled, unenrolled, courses_join, joined_courses = reg.courseListing(username.value)
    
    x1 = widgets.HTML(
    value="<b>Enrolled Courses</b>",
    )
    display(x1)
    display(enrolled)    
    show_course_dropdown = widgets.Dropdown(
    options = joined_courses)  
    show_course_button = widgets.Button(description="Show")
    show_course_button.on_click(show_course)
    display(show_course_dropdown)
    display(show_course_button)  
    
       
    x2 = widgets.HTML(
    value="<b>Join Courses</b>",
    )
    display(x2)
    columnList = ["course_id", "course_title", "course_rating", "Tutor", "Category"]
    display(unenrolled)
    reg.closeConnection()
    enroll = widgets.Dropdown(
    options = courses_join)    
    display (enroll)
    
    join_button = widgets.Button(description="Join")
    join_button.on_click(join_course)
    display(join_button)    

button.on_click(course_listing)




HTML(value='<H1>potions</H1><br>Rating: 5<br>Tutor: severus snape<br>Average Score: 63.00<br>')

HTML(value='<b>Documents</b>')

,Title
0,Potions Fall-17


HTML(value='<b>Videos</b>')

,Title
0,Intro to Potions


...

,Title,Description,Pass Mark,Scored,Result
0,Potions,Potions Exam,75.0,63,fail


HTML(value='<b>Reivews</b>')

,Student,Rating,Comment
0,Harry potter,3,bad


HTML(value='<b>Classmates</b>')

,Student
0,Harry potter
